In [1]:
# Load in the most common SMILE states
from smile.common import * 

# enter configuration variables here (including the listgen variables)
font_size = 75
ISI_dur = 0.5  # Inter-stimulus interval duration
ISI_jitter = 0.2  # Jitter for ISI
resp_keys = ['F', 'J']  # Response keys for old and new items
study_duration_time = 1.0 #Time study word appears on screen

# call the listgen code to create your blocks
# (you can copy it in here from the solution notebook)
# some useful imports
import random
import csv
from copy import deepcopy
import pickle

# Code to read in the pools
def read_and_shuffle(pool_file):
    """Read in and shuffle a pool."""
    # create a dictionary reader
    dr = csv.DictReader(open(pool_file, 'r'))

    # read in all the lines into a list of dicts
    pool = [l for l in dr]

    # shuffle it so that the we get new items each time
    random.shuffle(pool)
    
    # report out some pool info
    print(pool_file, len(pool))

    # return the shuffled pool
    return pool

# Config Section
pool_files = {'pos': 'pos_pool.csv',
              'neg': 'neg_pool.csv',
              'neu': 'neu_pool.csv'}

rep_conds = ['once', 'repeat', 'refresh']
val_conds = ['pos', 'neg', 'neu']

# what to show for refreshed items
ref_text = '+'

num_reps = 1
num_blocks = 3

# read all the pools into a dictionary
pools = {val: read_and_shuffle(pool_files[val])
         for val in val_conds}

# show the first 5 items of the pos pool
pools['pos'][:5]

# create the conds
# fully crossed with all combos of val and rep
conds = []
for val in val_conds:
    for rep in rep_conds:
        # I decided to call the repetition condition cond
        conds.append({'valence': val, 'cond': rep})
conds

# make a function for generating a block
# with a study and test list
def make_block():
    # loop and create the repeated conditions
    block_conds = []
    for i in range(num_reps):
        # extend the trials with copies of the conditions
        block_conds.extend(deepcopy(conds))

    # shuffle the conds for that block
    random.shuffle(block_conds)

    # loop over block conds and add items to study/test lists
    study_list = []
    test_list = []
    for cond in block_conds:
        # use the valence to grab study and test items
        study_item = pools[cond['valence']].pop()
        test_item = pools[cond['valence']].pop()

        # update with the cond info
        study_item.update(cond)
        test_item.update(cond)

        # add in relevant info for study and test
        study_item['pres_num'] = 1
        study_item['type'] = 'target'
        test_item['type'] = 'lure'
        test_item['pres_num'] = 1   # just so the keys match

        # append them to the respective lists
        # study item is added to both study and test
        study_list.append(study_item)
        test_list.append(study_item)
        test_list.append(test_item)

        # process the repetition and refresh conditions
        if cond['cond'] in ['refresh', 'repeat']:
            # copy the study item
            rep_item = deepcopy(study_item)

            # modify required values
            rep_item['pres_num'] = 2

            # change the description if refreshing
            if cond['cond'] == 'refresh':
                rep_item['description'] = ref_text

            # append it to the study list
            study_list.append(rep_item)
            
    # must shuffle the test list
    random.shuffle(test_list)
    
    # make a dictionary to return
    block = {'study': study_list, 'test': test_list}
    
    return block

# generate the proper number of blocks
blocks = []
for b in range(num_blocks):
    blocks.append(make_block())
    
blocks

# create an experiment instance
exp = Experiment(show_splash=False, resolution=(1024,768))

# YOUR CODE HERE TO BUILD THE STATE MACHINE
@Subroutine
def Instructions(self,text): #Initial study instructions
    Label(text=text, text_size=(500,None)) 
    with UntilDone():
        KeyPress()
    Wait(ISI_dur, jitter=ISI_jitter)
        
@Subroutine
def StudyPhase_Instructions(self,text): #INstructions prior to study phase
    Label(text=text, text_size=(500,None)) 
    with UntilDone():
        KeyPress()
    Wait(ISI_dur, jitter=ISI_jitter)
        
@Subroutine
def StudyPhase(self, block_num, trial_num, cur_trial): #code to present study words
    stim = Label(text=cur_trial['description'], font_size=font_size,duration=study_duration_time)
        
    Wait(ISI_dur, jitter=ISI_jitter)
    
    Log(cur_trial, name='study_log',
        stim_on=stim.appear_time
       ) 
    
@Subroutine
def TestPhase_Instructions(self,text): #instructions for test phase
    Label(text=text, text_size=(500,None)) 
    with UntilDone():
        KeyPress()
    Wait(ISI_dur, jitter=ISI_jitter)

@Subroutine
def TestPhase(self, block_num, trial_num, cur_trial): #code to present test words
    stim = Label(text=cur_trial['description'], font_size=font_size)
    with UntilDone():
        Wait(until=stim.appear_time)
        kp = KeyPress(keys=resp_keys)
    
    Wait(ISI_dur, jitter=ISI_jitter)
    
    Log(cur_trial, name='test_log',
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time
       )
    
@Subroutine
def FinInstructions(self,text): #test that appears when the participant is done
    Label(text=text, text_size=(500,None)) 
    with UntilDone():
        KeyPress()
    Wait(ISI_dur, jitter=ISI_jitter)
    
Instructions(text='Welcome to the recognition memory experiment! Your task is to indicate whether each item was part of the study list (OLD) or not (NEW). Use the "F" key for OLD items and the "J" key for NEW items. Press any key to begin.')
        
with Loop(blocks) as block:
        
    with If(block.i != 0): #to give the participant a break in between blocks if they are tired
        Label(text='You are now moving onto the next block. If you are tired, please take a break! Press any key to continue when you are ready.')
        with UntilDone():
            KeyPress()
        Wait(ISI_dur, jitter=ISI_jitter)
        
    StudyPhase_Instructions(text='These are some words for you to study. Please try to remember them the best you can. The words will automatically appear, and you do not need to press anything when they do. IF you see a "+" sign, try to refresh on the word you just saw before. Press any key to continue onto your study list.')
   
    with Loop(block.current['study']) as trial:
        StudyPhase(block.i,trial.i,trial.current)
    TestPhase_Instructions(text='Now, there will be another set of words. If you think you saw the word in the previous list, press "F". If you think the word is new, press "J". Press any key to continue onto your test list.')
    with Loop(block.current['test']) as trial:
        TestPhase(block.i,trial.i,trial.current)
    
    with If(block.i + 1 == num_blocks):
        Label(text='Great job! You are done with the experiment. Congratulations! Press any key to close the window.')
        with UntilDone():
            KeyPress()
        Wait(ISI_dur, jitter=ISI_jitter)
    
# run the experiment
exp.run()

[INFO   ] [Logger      ] Record log in C:\Users\lucas\.kivy\logs\kivy_23-12-20_0.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.gstreamer" 0.3.3
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.3.3
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.1
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.6.0
[INFO   ] [Kivy        ] v2.2.1
[INFO   ] [Kivy        ] Installed at "C:\Users\lucas\anaconda3\envs\smile\Lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "C:\Users\lucas\anaconda3\envs\smile\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 190 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sd

pos_pool.csv 301
neg_pool.csv 292
neu_pool.csv 208


[INFO   ] [Base        ] Start application main loop
[INFO   ] [GL          ] NPOT texture support is available
[INFO   ] [Base        ] Leaving application in progress...
